In [2]:
import select
import psycopg2
import psycopg2.extensions

field_names =  ['user_screen_name', 'user_name', 'created_at', 'sentiment', 'categories', 'text']

In [ ]:
from flask import Flask, render_template, jsonify
from flask_sockets import Sockets
from json import dumps
import gevent
import time
from gevent.pool import Pool

col_names = ['id_str', 'categories', 'in_reply_to_user_id', 'sentiment', 'text', 'created_at',
            'user_location','in_reply_to_status_id','in_reply_to_screen_name',
            'retweet_count','favorite_count','user_name','user_screen_name','user_id_str']

app = Flask(__name__)
sockets = Sockets(app)

@sockets.route('/twitter_stream')
def twitter_stream(ws):
    j = 0
    
    conn = psycopg2.connect(database="twitter", user="postgres")
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    curs = conn.cursor()
    curs.execute("LISTEN new_tweet;")
    
    while not ws.closed:
        conn.poll()
        t0 = time.time()
        if conn.notifies:
            notify = conn.notifies.pop(0)
            
            ws.send("{notification:\'new tweet\'}")
            j+=1
        if time.time() - t0 > 3600:
            print("1hr timeout")
            break
        else:
            gevent.sleep()
    conn.close()

@app.route('/data', methods=['GET'])
def provide_data():
    #https://twitter.com/statuses/ID
    conn = psycopg2.connect(database="twitter", user="postgres")
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    curs = conn.cursor()

    curs.execute("""select user_screen_name, user_name, 
                created_at, sentiment, categories, text
                from stream 
                where user_screen_name in 
                (
                    select tmp.user_screen_name
                    from (
                        select user_screen_name as user_screen_name, max(id_str) as max_id_str
                        from stream
                        group by user_screen_name
                    ) tmp
                    order by tmp.max_id_str desc limit 10
                );""")
    rec = curs.fetchall()
    conn.close()
    
    user_blocks = {v[0]:[] for v in rec}
    for row in rec:
        if len(user_blocks[row[0]]) < 5:
            user_blocks[row[0]] += [{k:v for k,v in zip(field_names, row)}]
    
    
    
    return jsonify({'data':user_blocks.values()})

@app.route('/')
def homepage():
    try:
        return render_template("index.html")
    except Exception, e:
        return str(e)

if __name__ == "__main__":
    from gevent import pywsgi
    from geventwebsocket.handler import WebSocketHandler
    server = pywsgi.WSGIServer(('', 8090), app, handler_class=WebSocketHandler, spawn=Pool())
    server.serve_forever()

In [ ]:
df